In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [69]:
url = "http://scholarworks.unist.ac.kr/browse-researcher"
data = {
    "sortType": "English",
    "page": 1,
    "offset": 0
}

In [70]:
## 데이터 정보를 보내지 않았다!
r = requests.get(url, params=data)
text = r.text
soup = BeautifulSoup(text, 'lxml')

In [100]:
li = soup.findAll("li")[16:76]
li[0:5]

[<li>
 <a href="researcher-profile?ep=2741">Ahn, Sangjoon
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2781">Amblard, Francois
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2641">An, Kwangjin
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=1881">Au, Tsz-Chiu
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2041">Bae, Hantaek
 
 						</a>
 </li>]

In [79]:
link = []
name = []

def makelink(pg):
    url = "http://scholarworks.unist.ac.kr/browse-researcher"
    data = {
        "sortType": "English",
        "page": pg,
        "offset": 0
    }
    r = requests.get(url, params=data)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    for i in soup.findAll("li")[16:76]:
        link.append(i.find("a")['href'])
        name.append(i.getText().replace("\n","").replace("\r","").replace("\t",""))

In [80]:
for i in range(5):
    makelink(i+1)

In [88]:
def makelink6():
    url = "http://scholarworks.unist.ac.kr/browse-researcher"
    data = {
        "sortType": "English",
        "page": 6,
        "offset": 0
    }
    r = requests.get(url, params=data)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    for i in soup.findAll("li")[16:28]:
        link.append(i.find("a")['href'])
        name.append(i.getText().replace("\n","").replace("\r","").replace("\t",""))

In [89]:
makelink6()

In [93]:
link[311], name[311]

('researcher-profile?ep=1302', 'Zhao, Bum Suk')

In [94]:
client = MongoClient()
db = client.aboutdepart

In [96]:
for i, k in zip(link,name):    
    document = {}
    document["link"] = i
    document["name"] = k
    db.scholar.insert_one(document)

In [97]:
db.scholar.count()

/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


312

In [98]:
Contents = db.scholar.find()
Contents[5]

{'_id': ObjectId('5bdfed2e99ba9f810182fd37'),
 'link': 'researcher-profile?ep=1249',
 'name': 'Bae, Joonbum'}

-------------------------------------

### db.scholar에서 저장한 link를 타고 url을 호출합니다.

In [101]:
url = "http://scholarworks.unist.ac.kr/"+str(link[0])
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

In [102]:
soup

<!DOCTYPE html>
<html>
<head>
<title>ScholarWorks: Researcher Page</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="DSpace 4.3" name="Generator"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="IE=Edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<!-- Facebook 공유 Thumbnail -->
<meta content="http://scholarworks.unist.ac.kr:80/image/background/facebook-thumb800.jpg" property="og:image"/>
<link href="http://scholarworks.unist.ac.kr:80/image/background/facebook-thumb800.jpg" rel="image_src"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/css/common.css" media="screen" rel="stylesheet" title="web2.0" type="text/css"/>
<script src="/static/js/jquery/jquery-1.10.2.min.js" type="text/javascript"></script>
<script src="/js/utils.js" type="text/javascript"></script>
<script src="/js/koasas.js" type=

In [169]:
soup.find('div',{'class' : "research_info_view"})

<div class="research_info_view">
<div class="researcher_idbox">
<ul>
<li>
<a href="http://www.researcherid.com/rid/R-2161-2016" id="R-2161-2016" target="_blank"><img alt="ResearcherID" class="tooltip" src="/image/common/research_icon02.gif" title="ResearcherID"/></a>
</li>
<li>
<a href="https://www.researchgate.net/profile/Sangjoon_Ahn2" id="https://www.researchgate.net/profile/Sangjoon_Ahn2" target="_blank"><img alt="ResearchGate" class="tooltip" src="/image/common/research_icon03.gif" title="ResearchGate"/></a>
</li>
<li>
<a href="http://orcid.org/0000-0003-1853-6439 " id="http://orcid.org/0000-0003-1853-6439 " target="_blank"><img alt="ORCiD" class="tooltip" src="/image/common/research_icon01.gif" title="ORCiD"/></a>
</li>
<li>
<a href="http://www.scopus.com/authid/detail.url?authorId=55614074100" id="http://www.scopus.com/authid/detail.url?authorId=55614074100" target="_blank"><img alt="Scopus" class="tooltip" src="/image/common/SCPID_icon.png" title="Scopus" width="34"/></a>
</li>

In [128]:
soup.find('div',{'class' : "research_info_view"}).findAll('dl')

[<dl>
 <dt>Department</dt>
 <dd>
 			School of Mechanical, Aerospace and Nuclear Engineering(기계항공및원자력공학부)
 			</dd>
 </dl>, <dl>
 <dt>Research Interests</dt>
 <dd>
 			Nuclear Fuel Performance Experiments &amp; Modeling, Radiation Interactions with Matter, Thermophysical Investigation of Nuclear Materials, Nuclear Non-Proliferation Technology
        		</dd>
 </dl>, <dl>
 <dt>Lab</dt>
 <dd>UNIST RAdioactive NUclear Materials (URANUM) Laboratory</dd>
 </dl>, <dl>
 <dt>Website</dt>
 <dd><a href="http://uranum.unist.ac.kr" target="_blank">http://uranum.unist.ac.kr</a></dd>
 </dl>]

In [131]:
#department
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[0].find('dd').getText()

'\r\n\t\t\tSchool of Mechanical, Aerospace and Nuclear Engineering(기계항공및원자력공학부)\r\n\t\t\t'

In [133]:
#Research Interest
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[1].find('dd').getText()

'\r\n\t\t\tNuclear Fuel Performance Experiments & Modeling, Radiation Interactions with Matter, Thermophysical Investigation of Nuclear Materials, Nuclear Non-Proliferation Technology\r\n       \t\t'

In [135]:
#Lab
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[2].find('dd').getText()

'UNIST RAdioactive NUclear Materials (URANUM) Laboratory'

In [136]:
#Website
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[3].find('dd').getText()

'http://uranum.unist.ac.kr'

### 학술지 정보

In [118]:
soup.find('div',{'class' : "researcher_idbox"})

<div class="researcher_idbox">
<ul>
<li>
<a href="http://www.researcherid.com/rid/R-2161-2016" id="R-2161-2016" target="_blank"><img alt="ResearcherID" class="tooltip" src="/image/common/research_icon02.gif" title="ResearcherID"/></a>
</li>
<li>
<a href="https://www.researchgate.net/profile/Sangjoon_Ahn2" id="https://www.researchgate.net/profile/Sangjoon_Ahn2" target="_blank"><img alt="ResearchGate" class="tooltip" src="/image/common/research_icon03.gif" title="ResearchGate"/></a>
</li>
<li>
<a href="http://orcid.org/0000-0003-1853-6439 " id="http://orcid.org/0000-0003-1853-6439 " target="_blank"><img alt="ORCiD" class="tooltip" src="/image/common/research_icon01.gif" title="ORCiD"/></a>
</li>
<li>
<a href="http://www.scopus.com/authid/detail.url?authorId=55614074100" id="http://www.scopus.com/authid/detail.url?authorId=55614074100" target="_blank"><img alt="Scopus" class="tooltip" src="/image/common/SCPID_icon.png" title="Scopus" width="34"/></a>
</li>
</ul>
</div>

In [163]:
for i in soup.find('div',{'class' : "researcher_idbox"}).findAll('a'):
    print(i['href'])

http://www.researcherid.com/rid/R-2161-2016
https://www.researchgate.net/profile/Sangjoon_Ahn2
http://orcid.org/0000-0003-1853-6439 
http://www.scopus.com/authid/detail.url?authorId=55614074100


In [168]:
soup.find('div',{'class' : "researcher_idbox"}).findAll('a')[-1]['href'].replace("http://www.scopus.com/authid/detail.url?authorId=","")

'55614074100'

## Final